# Schemata anwenden

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = (SparkSession
         .builder
         .appName("schemata")
         .master("local[4]")
         .config("spark.dynamicAllocation.enabled", "false")
         .config("spark.sql.adaptive.enabled", "false")
         .getOrCreate()
        )
sc = spark.sparkContext
spark

In [6]:
from IPython.display import *
display(HTML("<style>pre { white-space: pre !important; }</style>"))

## Schemalos

In [7]:
yellow_taxi_df = spark.read.option("header", True).csv("YellowTaxis_202210.csv.gz")
yellow_taxi_df.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)
 |-- airport_fee: string (nullable = true)



* In Spark UI checken, dass ein Job erzeugt wurde. 
* Aber eigentlich ist das Lesen doch eine Transformation?
* Wieso dann ein Job?
* Was zum SQL/DataFrame Tab in Spark UI sagen
* Zeigen dass die Datentypen nocht nicht korrekt sind.


## Automatisches Schema

In [8]:
yellow_taxi_df = spark.read.option("header", True).option("inferSchema", True).csv("YellowTaxis_202210.csv.gz")
yellow_taxi_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



### Vor- und Nachteile des Automatischen erkennen von Schemata

#### Pro

* gut während der Entwicklung um sich mit den Daten vertraut zu machen
* kein Aufwand für das Schreiben von Schemata notwendig

#### Cons

* Fehler in den Daten führen zu falschem Schema
* dauert lange

#### Deswegen

Für produktive Spark Applikationen Schemata besser manuell angeben


## Manuelle Schemata

In [10]:
%less schemata.py

from pyspark.sql.types import *

yellow_taxi_schema  =  (  StructType
                        ([ 
                            StructField("VendorId"               , IntegerType()   , True),
                            StructField("tpep_pickup_datetime"   , TimestampType() , True),
                            StructField("tpep_dropoff_datetime"  , TimestampType() , True),
                            StructField("passenger_count"        , DoubleType()    , True),
                            StructField("trip_distance"          , DoubleType()    , True),
                            StructField("RatecodeID"             , DoubleType()    , True),
                            StructField("store_and_fwd_flag"     , StringType()    , True),
                            StructField("PULocationID"           , IntegerType()   , True),
                            StructField("DOLocationID"           , IntegerType()   , True),
                            StructField("payment_type"           , Integer

In [13]:
import schemata
yellow_taxi_df = (
    spark
        .read
        .option("header", True)
        .schema(schemata.yellow_taxi_schema)
        .csv("YellowTaxis_202210.csv.gz")
)

In [12]:
yellow_taxi_df.printSchema()

root
 |-- VendorId: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In der Spark UI validieren, dass wirklich kein Job dafür gestartet wurde.

### Nun mit einem Json File

In [15]:
taxi_bases_df = spark.read.json("TaxiBases.json")

In [16]:
taxi_bases_df.show()

AnalysisException: Since Spark 2.3, the queries from raw JSON/CSV files are disallowed when the
referenced columns only include the internal corrupt record column
(named _corrupt_record by default). For example:
spark.read.schema(schema).csv(file).filter($"_corrupt_record".isNotNull).count()
and spark.read.schema(schema).csv(file).select("_corrupt_record").show().
Instead, you can cache or save the parsed results and then send the same query.
For example, val df = spark.read.schema(schema).csv(file).cache() and then
df.filter($"_corrupt_record".isNotNull).count().

In [17]:
taxi_bases_df = spark.read.option("multiline", True).json("TaxiBases.json")

In [20]:
taxi_bases_df.show(3, truncate=False)

+--------------------------------------------+----------+-------------------------------+------------------------------------------------+--------------+------------+----------------+--------+---------------------------+
|Address                                     |Date      |Entity Name                    |GeoLocation                                     |License Number|SHL Endorsed|Telephone Number|Time    |Type of Base               |
+--------------------------------------------+----------+-------------------------------+------------------------------------------------+--------------+------------+----------------+--------+---------------------------+
|{636, NEW YORK, 10001, NY, WEST   28 STREET}|08/15/2019|VIER-NY,LLC                    |{40.75273, (40.75273, -74.006408), -74.006408}  |B02865        |No          |6466657536      |18:03:31|BLACK CAR BASE             |
|{131, BRONX, 10468, NY, KINGSBRIDGE ROAD}   |08/15/2019|VETERANS RADIO DISPATCHER CORP.|{40.86927, (40.86927, -73.9

In [21]:
taxi_bases_df.printSchema()

root
 |-- Address: struct (nullable = true)
 |    |-- Building: string (nullable = true)
 |    |-- City: string (nullable = true)
 |    |-- Postcode: long (nullable = true)
 |    |-- State: string (nullable = true)
 |    |-- Street: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Entity Name: string (nullable = true)
 |-- GeoLocation: struct (nullable = true)
 |    |-- Latitude: double (nullable = true)
 |    |-- Location: string (nullable = true)
 |    |-- Longitude: double (nullable = true)
 |-- License Number: string (nullable = true)
 |-- SHL Endorsed: string (nullable = true)
 |-- Telephone Number: long (nullable = true)
 |-- Time: string (nullable = true)
 |-- Type of Base: string (nullable = true)



In [24]:
taxi_bases_df = spark.read.option("multiline", True).schema(schemata.taxi_bases_schema).json("TaxiBases.json")
taxi_bases_df.printSchema()
taxi_bases_df.show(truncate=False)

root
 |-- License Number: string (nullable = true)
 |-- Entity Name: string (nullable = true)
 |-- Telephone Number: long (nullable = true)
 |-- SHL Endorsed: string (nullable = true)
 |-- Type of Base: string (nullable = true)
 |-- Address: struct (nullable = true)
 |    |-- Building: string (nullable = true)
 |    |-- Street: string (nullable = true)
 |    |-- City: string (nullable = true)
 |    |-- State: string (nullable = true)
 |    |-- Postcode: string (nullable = true)
 |-- GeoLocation: struct (nullable = true)
 |    |-- Latitude: string (nullable = true)
 |    |-- Longitude: string (nullable = true)
 |    |-- Location: string (nullable = true)

+--------------+--------------------------------------+----------------+------------+---------------------------+-----------------------------------------------------------+------------------------------------------------+
|License Number|Entity Name                           |Telephone Number|SHL Endorsed|Type of Base               |A

Was nimmst Du aus dieser Lektion mit?

Du kannst ein Schema automatisch erkennen, aber das dauert teils lange und kann auch zu nicht korrekten Ergebnissen führen. Deswegen ist es in der Praxis empfehleungswert Schemata manuell zu erzeugen und anzugeben.
